In [ ]:
import math

In [ ]:
'''
Audio stats dictionary contains these params:

 * packetLoss: 0-100%
 * bitrate: bps
 * roundTripTime: ms
 * bufferDelay: ms
 * fec: boolean (ony used for audio)
 * dtx: boolean (ony used for audio)

Video stats dictionary contains these params:

 * packetLoss: 0-100%
 * bitrate: bps
 * roundTripTime: ms
 * bufferDelay: ms
 * codec: opus / vp8 / vp9 / h264 (only used for video) **EDIT THIS --> maybe add AV1 codec
 * width: number; Resolution of the video received
 * expectedWidth: number; Resolution of the rendering widget
 * height: number; Resolution of the video received
 * expectedHeight: number; Resolution of the rendering widget
 * frameRate: number; FrameRate of the video received
 * expectedFrameRate: number; FrameRate of the video source
'''

def clamp(val, minimum, maximum):
	if val >= minimum:
		if val <= maximum:
			return val
		else:
			return maximum
	else:
		return minimum
	

def score_audio(audio_stats):

	R0 = 100 #initial rating (starts at max 100)

	delay = 20 + audio_stats['bufferDelay'] + audio_stats['roundTripTime'] / 2 

	PL = audio_stats['packetLoss'] #Packet Loss

	#Equiptment Impairment 
	if audio_stats['dtx']:
		EI = 8
	else:
		value = 55 - 4.6 * math.log(audio_stats['bitrate'])
		EI = clamp(value, 0, 30)
	
	#Packet Loss Robustness Factor
	if audio_stats['fec']:
		PLRF = 20
	else:
		PLRF = 10   
	
	PLI = EI + (100 - EI) * (PL / (PL + PLRF)) #Packet Loss Impairment 
	
	#Delay Impairment
	if delay > 150:
		DI = delay * 0.03 + (0.1 * (delay - 150))
	else:
		DI = delay * 0.03
		
	R = clamp(R0 - PLI - DI, 0, 100) #Rating
	
	MOS_raw = 1 + 0.035 * R + (R * (R - 60) * (100 - R) * 7) / 1000000 
	MOS = clamp(math.round(MOS * 100) / 100, 1, 5) #MOS score bounded between 1-5 to 2 d.p.
	
	return MOS

def score_video(video_stats):

		






